## Data Cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#loaded data as well as airport data with information about the airports from https://datahub.io/core/airport-codes 

df = pd.read_csv('data/Train.csv')
airport_data = pd.read_csv('data/airport_data.csv', dtype={'continent': 'string'}, na_values=[''])
airport_data['continent'].fillna('NA', inplace=True)
df.columns = df.columns.str.lower()

In [3]:
#delete rows with status DEL, SCH, RTR, DEP
df = df[~df['status'].isin(['DEL', 'SCH', 'RTR', 'DEP'])]

In [4]:
#create new column airline
df['airline'] = df['ac'].apply(lambda x: x[:2])

In [5]:
# Convert datop (date of flight) to datetime
df['datop'] = pd.to_datetime(df['datop'], format="%Y-%m-%d", errors='coerce')

# Extract time from scheduled time departure (std) and confert to datetime
df['std'] = pd.to_datetime(df['std']).dt.strftime('%H:%M:%S')

# Extract time from scheduled time arrival (sta) and convert to datetime
df['sta'] = pd.to_datetime(df['sta'], format='%Y-%m-%d %H.%M.%S').dt.strftime('%H:%M:%S')

In [6]:
df.head()

,id,datop,fltid,depstn,arrstn,std,sta,status,ac,target,airline
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,10:30:00,12:55:00,ATA,TU 32AIMN,260.0,TU
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,15:05:00,16:55:00,ATA,TU 31BIMO,20.0,TU
2,train_id_2,2016-01-16,TU 0214,TUN,IST,04:10:00,06:45:00,ATA,TU 32AIMN,0.0,TU
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,14:10:00,17:00:00,ATA,TU 736IOK,0.0,TU
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,14:30:00,15:50:00,ATA,TU 320IMU,22.0,TU


In [7]:
# Extract day of the week
df['day_of_week'] = df['datop'].dt.day_name()

In [8]:
#Extract morning, midday, afternoon, evening, night from scheduled time departure (std) and scheduled time arrival (sta)
def get_time_of_day(time_str):
    hour = int(time_str.split(':')[0])
    if 0 <= hour < 6:
        return 'night'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 14:
        return 'midday'
    elif 14 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

df['time_day_std'] = df['std'].apply(get_time_of_day)
df['time_day_sta'] = df['sta'].apply(get_time_of_day)



In [9]:
df.head()

,id,datop,fltid,depstn,arrstn,std,sta,status,ac,target,airline,day_of_week,time_day_std,time_day_sta
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,10:30:00,12:55:00,ATA,TU 32AIMN,260.0,TU,Sunday,morning,midday
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,15:05:00,16:55:00,ATA,TU 31BIMO,20.0,TU,Wednesday,afternoon,afternoon
2,train_id_2,2016-01-16,TU 0214,TUN,IST,04:10:00,06:45:00,ATA,TU 32AIMN,0.0,TU,Saturday,night,morning
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,14:10:00,17:00:00,ATA,TU 736IOK,0.0,TU,Sunday,afternoon,evening
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,14:30:00,15:50:00,ATA,TU 320IMU,22.0,TU,Sunday,afternoon,afternoon


In [10]:
# Searched for missing information in airport_data and added to airport_data file
airport_info = [
    {'iata_code': 'AMM', 'continent': 'AS', 'iso_country': 'JO', 'type': 'large_airport'},
    {'iata_code': 'BEY', 'continent': 'AS', 'iso_country': 'LB', 'type': 'large_airport'},
    {'iata_code': 'DOH', 'continent': 'AS', 'iso_country': 'QA', 'type': 'large_airport'},
    {'iata_code': 'EBL', 'continent': 'AS', 'iso_country': 'IQ', 'type': 'large_airport'},
    {'iata_code': 'IEV', 'continent': 'EU', 'iso_country': 'UA', 'type': 'medium_airport'},
    {'iata_code': 'JED', 'continent': 'AS', 'iso_country': 'SA', 'type': 'large_airport'},
    {'iata_code': 'KBP', 'continent': 'EU', 'iso_country': 'UA', 'type': 'large_airport'},
    {'iata_code': 'KRR', 'continent': 'EU', 'iso_country': 'RU', 'type': 'large_airport'},
    {'iata_code': 'LED', 'continent': 'EU', 'iso_country': 'RU', 'type': 'large_airport'},
    {'iata_code': 'MED', 'continent': 'AS', 'iso_country': 'SA', 'type': 'medium_airport'},
    {'iata_code': 'SKX', 'continent': 'EU', 'iso_country': 'RU', 'type': 'small_airport'},
    {'iata_code': 'SVO', 'continent': 'EU', 'iso_country': 'RU', 'type': 'large_airport'},
    {'iata_code': 'SXF', 'continent': 'EU', 'iso_country': 'DE', 'type': 'large_airport'},
    {'iata_code': 'VKO', 'continent': 'EU', 'iso_country': 'RU', 'type': 'large_airport'},
    {'iata_code': 'VOG', 'continent': 'EU', 'iso_country': 'RU', 'type': 'medium_airport'}
]

# Convert airport_info to DataFrame
new_info_df = pd.DataFrame(airport_info)

# Update existing rows or append new rows
for index, row in new_info_df.iterrows():
    iata_code = row['iata_code']
    if iata_code in airport_data['iata_code'].values:
        airport_data.loc[airport_data['iata_code'] == iata_code, ['continent', 'iso_country', 'type']] = row[['continent', 'iso_country', 'type']]
    else:
        airport_data = pd.concat([airport_data, row.to_frame().T], ignore_index=True)

airport_data.head(20)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,icao_code,iata_code,gps_code,local_code,coordinates
0,00A,heliport,Total RF Heliport,11.0,NA,US,US-PA,Bensalem,NaN,NaN,K00A,00A,"40.070985, -74.933689"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NA,US,US-KS,Leoti,NaN,NaN,00AA,00AA,"38.704022, -101.473911"
2,00AK,small_airport,Lowell Field,450.0,NA,US,US-AK,Anchor Point,NaN,NaN,00AK,00AK,"59.947733, -151.692524"
3,00AL,small_airport,Epps Airpark,820.0,NA,US,US-AL,Harvest,NaN,NaN,00AL,00AL,"34.86479949951172, -86.77030181884766"
4,00AN,small_airport,Katmai Lodge Airport,80.0,NA,US,US-AK,King Salmon,NaN,NaN,00AN,00AN,"59.093287, -156.456699"
5,00AS,small_airport,Fulton Airport,1100.0,NA,US,US-OK,Alex,NaN,NaN,00AS,00AS,"34.9428028, -97.8180194"
6,00AZ,small_airport,Cordes Airport,3810.0,NA,US,US-AZ,Cordes,NaN,NaN,00AZ,00AZ,"34.305599212646484, -112.16500091552734"
7,00CA,small_airport,Goldstone (GTS) Airport,3038.0,NA,US,US-CA,Barstow,NaN,NaN,00CA,00CA,"35.35474, -116.885329"
8,00CL,small_airport,Williams Ag Airport,87.0,NA,US,US-CA,Biggs,NaN,NaN,00CL,00CL,"39.427188, -121.763427"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350.0,NA,US,US-CA,Pine Valley,NaN,NaN,00CN,00CN,"32.7273736, -116.4597417"


In [11]:
#select interesting columns from airport_data
airport_data = airport_data[['continent', 'iso_country', 'type', 'iata_code']]

In [12]:
#merge airport data with train data
df = df.merge(airport_data, left_on='depstn', right_on='iata_code', how='left')
df = df.merge(airport_data, left_on='arrstn', right_on='iata_code', how='left', suffixes=('_dep', '_arr'))
df = df.drop(columns=['iata_code_dep', 'iata_code_arr'])
df.head()

,id,datop,fltid,depstn,arrstn,std,sta,status,ac,target,airline,day_of_week,time_day_std,time_day_sta,continent_dep,iso_country_dep,type_dep,continent_arr,iso_country_arr,type_arr
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,10:30:00,12:55:00,ATA,TU 32AIMN,260.0,TU,Sunday,morning,midday,AF,MA,large_airport,AF,TN,large_airport
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,15:05:00,16:55:00,ATA,TU 31BIMO,20.0,TU,Wednesday,afternoon,afternoon,EU,IT,large_airport,AF,TN,large_airport
2,train_id_2,2016-01-16,TU 0214,TUN,IST,04:10:00,06:45:00,ATA,TU 32AIMN,0.0,TU,Saturday,night,morning,AF,TN,large_airport,EU,TR,large_airport
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,14:10:00,17:00:00,ATA,TU 736IOK,0.0,TU,Sunday,afternoon,evening,AF,TN,medium_airport,EU,FR,medium_airport
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,14:30:00,15:50:00,ATA,TU 320IMU,22.0,TU,Sunday,afternoon,afternoon,AF,TN,large_airport,AF,DZ,large_airport


In [13]:
# Make new features with DP Country = Arr Country
df['is_same_country'] = np.where(df['iso_country_dep'].isna() | df['iso_country_arr'].isna(), np.nan, df['iso_country_dep'] == df['iso_country_arr'])

In [14]:
df.drop(columns=['datop', 'fltid', 'depstn', 'arrstn', 'std', 'sta', 'ac', 'status'], inplace=True)

In [15]:
df.head()

,id,target,airline,day_of_week,time_day_std,time_day_sta,continent_dep,iso_country_dep,type_dep,continent_arr,iso_country_arr,type_arr,is_same_country
0,train_id_0,260.0,TU,Sunday,morning,midday,AF,MA,large_airport,AF,TN,large_airport,0.0
1,train_id_1,20.0,TU,Wednesday,afternoon,afternoon,EU,IT,large_airport,AF,TN,large_airport,0.0
2,train_id_2,0.0,TU,Saturday,night,morning,AF,TN,large_airport,EU,TR,large_airport,0.0
3,train_id_3,0.0,TU,Sunday,afternoon,evening,AF,TN,medium_airport,EU,FR,medium_airport,0.0
4,train_id_4,22.0,TU,Sunday,afternoon,afternoon,AF,TN,large_airport,AF,DZ,large_airport,0.0


In [16]:
df.to_csv('data/train_cleaned.csv', index=False)

### interesting features would be (need to impute some rows, missing values):
- iso_country_dep
- iso_country_arr
- continent_arr
- continent_dep
- is_same_country
- type_dep
- type_arr
- days of week
- airline
